# Processing workflow example

In [76]:
from rime.process_config import *
from rime.rime_functions import *
from rime.utils import *
from rime.core import GWLPathway

import dask
import dask.dataframe as dd
# from dask.diagnostics import ProgressBar
from dask.diagnostics import Profiler, ResourceProfiler, CacheProfiler, ProgressBar
from dask.distributed import Client

# from dask.distributed import Client # uncomment this to open Dask client to see performance
import glob
import numpy as np
import pandas as pd
import pyam
import re
import time
import xarray as xr
import yaml

In [77]:
# Load a yaml file that will help select multiple climate indicator files (not necessarily needed)
with open("indicator_params.yml", "r") as f:
    params = yaml.full_load(f)

In [78]:
dask.config.set(scheduler="processes")
dask.config.set(num_workers=num_workers)
client = Client()  # uncomment this to open Dask client

/home/byers/venvs/rimeclasses/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 42673 instead
  warnings.warn(


To see the Dask dasboard, open http://localhost:8787/status in your browser

### Import scenarios data

Load an IAMC scenarios dataset.  We can use the one that comes with the package.
Decide whether using global mean temperature or CO2 mode.  
Assign SSPs if missing and fix duplicate temperatures.  

In [79]:
os.getcwd()

'/home/byers/rime/rime'

In [80]:
df_scens_in = pyam.IamDataFrame('test_data/emissions_temp_AR6_small.xlsx')
# dft = df_scens_in.filter(variable=temp_variable) # temp_variable as defined in the config file, = 'AR6 climate diagnostics|Surface Temperature (GSAT)|MAGICCv7.5.3|50.0th Percentile'

# # Replace & fill missing SSP scenario allocation
# dft = ssp_helper(dft, ssp_meta_col="Ssp_family", default_ssp="SSP2")
# dft = GWLPathway('test_data/emissions_temp_AR6_small.xlsx', temperature_variable='AR6 climate diagnostics|Surface Temperature (GSAT)|MAGICCv7.5.3|50.0th Percentile'
                )

pyam.core - INFO: Reading file test_data/emissions_temp_AR6_small.xlsx
/home/byers/venvs/rimeclasses/lib/python3.10/site-packages/pyam/core.py:2692: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  index = pd.unique(index)
pyam.core - INFO: Reading meta indicators
/home/byers/venvs/rimeclasses/lib/python3.10/site-packages/pyam/core.py:2692: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  index = pd.unique(index)


AR6 climate diagnostics|Surface Temperature (GSAT)|MAGICCv7.5.3|50.0th Percentile
Temperature_variable(s) provided: AR6 climate diagnostics|Surface Temperature (GSAT)|MAGICCv7.5.3|50.0th Percentile


/home/byers/venvs/rimeclasses/lib/python3.10/site-packages/pyam/core.py:2692: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  index = pd.unique(index)


### Build a dataset to compare SSPs and IMPs

In [85]:
scenarios = ["SSP1-26", "SSP2-45", "SSP3-Baseline", "SSP5-Baseline"]
variable = (
    "AR6 climate diagnostics|Surface Temperature (GSAT)|MAGICCv7.5.3|50.0th Percentile"
)
ssps = df_scens_in.filter(scenario=scenarios, model="IMAGE*")#, variable=variable)
imps = df_scens_in.filter(
    IMP_marker=["CurPol", "ModAct", "SP", "GS", "Neg"])#, variable=variable)  # ,'Ren-2.0','Neg-2.0'
ssps_imp = ssps.append(imps)
# ssps_imp = ssp_helper(ssps_imp, ssp_meta_col="Ssp_family", default_ssp="SSP2")
ssps_imp = GWLPathway(ssps_imp, temperature_variable=variable)

AR6 climate diagnostics|Surface Temperature (GSAT)|MAGICCv7.5.3|50.0th Percentile
Temperature_variable(s) provided: AR6 climate diagnostics|Surface Temperature (GSAT)|MAGICCv7.5.3|50.0th Percentile


/home/byers/venvs/rimeclasses/lib/python3.10/site-packages/pyam/core.py:2692: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  index = pd.unique(index)
/home/byers/venvs/rimeclasses/lib/python3.10/site-packages/pyam/core.py:2692: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  index = pd.unique(index)
/home/byers/venvs/rimeclasses/lib/python3.10/site-packages/pyam/core.py:2692: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  index = pd.unique(index)
/home/byers/venvs/rimeclasses/lib/python3.10/site-packages/pyam/core.py:2692: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  index = p

# Generate some maps

## Example 1: multiple IAM scenarios, 1 climate indicator

In [7]:
# Requires some configuration - need to download the data from https://zenodo.org/records/10212339, and set the file paths in the file process_config.py

In [8]:

# ind = "pr"
# var = "pr"
# short = "r10"  # params["indicators"][ind][var]["short_name"]
ssp = "ssp2"
ftype = "abs"

In [9]:
# published data

# # Prepare data
# impact_data_dir = 'test_data//'
# files = glob.glob(f"{impact_data_dir}/{var}*{short}_*{ftype}.nc4")
# mapdata = xr.open_mfdataset(
#     files, preprocess=remove_ssp_from_ds, combine="nested", concat_dim="gmt"
# )
# mapdata = xr.concat([mapdata[x] for x in mapdata.data_vars], dim='gmt').to_dataset(name=f'{var}_{short}')
# mapdata = tidy_mapdata(mapdata)
# mapdata = mapdata.sortby("gmt").drop_duplicates("gmt")

In [10]:
# impact_data_dir = '/mnt/c/users/byers/iiasa/ECE.prog - Documents/Research Theme - NEXUS/Hotspots_Explorer_2p0/rcre_testing/rime_testdata'
# files = glob.glob(f"{impact_data_dir}/*{var}*{short}_*{ftype}.nc4")
# files

In [123]:
from rime.core import RasterArray
impact_data_dir = 'test_data'
ind = 'dri_qtot'
files = glob.glob(f"{impact_data_dir}/*{ind}_*{ftype}.nc4")

mapdata = RasterArray(files)

FrozenMappingWarningOnValuesAccess({'lon': 720, 'lat': 360, 'gwl': 5})


/home/byers/rime/rime/core.py:113: UserWarning: rename 'threshold' to 'gwl' does not create an index anymore. Try using swap_dims instead or use set_index after rename to create an indexed coordinate.
  self.dataset = self.dataset.rename({"gmt": "gwl"}) if "gmt" in self.dataset.dims else self.dataset


In [111]:
# Prepare data (using split_files on sharepoint)
# impact_data_dir = '/mnt/c/users/byers/iiasa/ECE.prog - Documents/Research Theme - NEXUS/Hotspots_Explorer_2p0/rcre_testing/rime_testdata'
impact_data_dir = 'test_data'
# indicators = ['dri','iavar']
ind = 'dri_qtot'

files = glob.glob(f"{impact_data_dir}/*{ind}_*{ftype}.nc4")
mapdata = xr.open_mfdataset(
    files, preprocess=remove_ssp_from_ds, combine="nested", concat_dim="gwl"
)


mapdata
# mapdata = xr.concat([mapdata[x] for x in mapdata.data_vars], dim='gwl').to_dataset(name=f'{var}_{short}')
# mapdata = tidy_mapdata(mapdata)
# mapdata = mapdata.sortby("gwl").drop_duplicates("gwl")

<xarray.Dataset> Size: 10MB
Dimensions:    (lon: 720, lat: 360, gwl: 5)
Coordinates:
  * lon        (lon) float32 3kB -179.8 -179.2 -178.8 ... 178.8 179.2 179.8
  * lat        (lat) float32 1kB 89.75 89.25 88.75 ... -88.75 -89.25 -89.75
    quantile   float64 8B 0.9
    stats      object 8B 'mean'
    threshold  (gwl) float64 40B 2.0 3.0 1.2 2.5 1.5
Dimensions without coordinates: gwl
Data variables:
    dri_qtot   (gwl, lat, lon) float64 10MB dask.array<chunksize=(1, 360, 720), meta=np.ndarray>

In [124]:
print("Test multiple IAM scenarios, 1 indicator")

start = time.time()

# Run function
# Using dask for small numbers of scenarios ~<10 can be slower due to spawning of workers.
# use_dask=True currently broken 
map_out_MS = map_transform_gwl_wrapper(
    ssps_imp,
    mapdata,
    years,
    caution_checks=True,
    use_dask=False,
    include_orig_gwl=False,
    gwl_name="gwl",
    drawdown_max=0.15,
    temp_min=1.2,
    temp_max=3.0,
    interpolation=0.01,
)

comp = dict(zlib=True, complevel=5)
encoding = {var: comp for var in map_out_MS.data_vars}
filename = f"scenario_maps_multiscenario_{ftype}_test_notebook.nc"
map_out_MS.to_netcdf(filename, encoding=encoding)

print("FINISHED Test multiple scenarios, 1 indicator")
print(f"{time.time()-start}")

Test multiple IAM scenarios, 1 indicator


AttributeError: 'GWLPathway' object has no attribute 'index'

In [14]:
map_out_MS

<xarray.Dataset> Size: 336MB
Dimensions:                                      (lon: 720, lat: 360, year: 18)
Coordinates:
  * lon                                          (lon) float32 3kB -179.8 ......
  * lat                                          (lat) float32 1kB 89.75 ... ...
  * year                                         (year) int64 144B 2015 ... 2100
Data variables:
    COFFEE_1_1_EN_NPi2020_400f_lowBECCS          (lat, lon, year) float64 37MB dask.array<chunksize=(360, 720, 1), meta=np.ndarray>
    GCAM_5_3_NGFS2_Current Policies              (lat, lon, year) float64 37MB dask.array<chunksize=(360, 720, 1), meta=np.ndarray>
    IMAGE_3_0_EN_INDCi2030_3000f                 (lat, lon, year) float64 37MB dask.array<chunksize=(360, 720, 1), meta=np.ndarray>
    IMAGE_3_0_1_SSP1-26                          (lat, lon, year) float64 37MB dask.array<chunksize=(360, 720, 1), meta=np.ndarray>
    IMAGE_3_0_1_SSP2-45                          (lat, lon, year) float64 37MB dask.array<chunksize=(360, 720, 2), meta=np.ndarray>
    IMAGE_3_0_1_SSP3-Baseline                    (lat, lon, year) float64 37MB dask.array<chunksize=(360, 720, 2), meta=np.ndarray>
    IMAGE_3_0_1_SSP5-Baseline                    (lat, lon, year) float64 37MB dask.array<chunksize=(360, 720, 2), meta=np.ndarray>
    REMIND-MAgPIE_2_1-4_2_SusDev_SDP-PkBudg1000  (lat, lon, year) float64 37MB dask.array<chunksize=(360, 720, 1), meta=np.ndarray>
    WITCH_5_0_CO_Bridge                          (lat, lon, year) float64 37MB dask.array<chunksize=(360, 720, 1), meta=np.ndarray>
Attributes:
    indicator:  dri_qtot

## Example 2: 1 scenario, multiple indicators


### Build combined dataset with multiple indicators

In [15]:
# Doesn't work -0
# AttributeError: 'DataArray' object has no attribute 'rename_vars'
# lines 230-233 in map_transform_gmt

gwl_name = "gwl"
print("Test 1 scenario, multiple indicators")
start = time.time()
ssp = "ssp2"
mapdata = xr.Dataset()

# impact_data_dir = '/mnt/p/watxene/ISIMIP_postprocessed/cse/split_files_fixed'
impact_data_dir = 'test_data'
indicators = ['dri','iavar']
for ind in indicators:

    print(ind)
    files = glob.glob(f"{impact_data_dir}/ISI*{ind}*_{ftype}*.nc4")
        
    a = xr.open_mfdataset(
            files, preprocess=remove_ssp_from_ds, combine="nested", concat_dim="gwl")
    for x in a.data_vars:
        mapdata[x] = a[x]


mapdata = tidy_mapdata(mapdata)
mapdata

Test 1 scenario, multiple indicators
dri
iavar


/home/byers/rime/rime/utils.py:121: UserWarning: rename 'threshold' to 'gwl' does not create an index anymore. Try using swap_dims instead or use set_index after rename to create an indexed coordinate.
  mapdata = mapdata.rename({"threshold": "gwl"})


<xarray.Dataset> Size: 16MB
Dimensions:     (lon: 720, lat: 360, gwl: 5)
Coordinates:
  * lon         (lon) float32 3kB -179.8 -179.2 -178.8 ... 178.8 179.2 179.8
  * lat         (lat) float32 1kB 89.75 89.25 88.75 ... -88.75 -89.25 -89.75
  * gwl         (gwl) float64 40B 1.2 1.5 2.0 2.5 3.0
Data variables:
    dri_qtot    (gwl, lat, lon) float64 10MB dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
    iavar_qtot  (gwl, lat, lon) float32 5MB dask.array<chunksize=(1, 360, 720), meta=np.ndarray>

In [16]:
os.getcwd()

'/home/byers/rime/rime'

### Run transformation (12 impact indicators) (faster with dask=False)

In [17]:
map_out_MI = map_transform_gwl_wrapper(
    ssps_imp.filter(IMP_marker="CurPol"),
    mapdata,
    years,
    use_dask=False,
    include_orig_gwl=False,
    gwl_name="gwl",
    drawdown_max=0.15,
    temp_min=1.2,
    temp_max=3.0,
    interpolation=0.01,
)

comp = dict(zlib=True, complevel=5)
encoding = {var: comp for var in map_out_MI.data_vars}
filename = f"scenario_maps_multiindicator_{ftype}_test_notebook.nc"
# map_out_MI.to_netcdf(filename, encoding=encoding)

print("FINISHED 1 scenario, multiple indicators")
print(f"{time.time()-start}")

Single scenario mode, multiple indicators possible
Warning! Max temperature above 3.0°C  3.05, data thereafter not possible
Warning! Min temperature below 1.2°C  1.12, data before not possible
Warning! Provided scenario temperatures are higher than the available climate impacts GWLs. Setting temp_max to <xarray.DataArray 'gwl' ()> Size: 8B
array(3.)
Warning! Provided scenario temperatures are lower than the available climate impacts GWLs. Setting temp_min to <xarray.DataArray 'gwl' ()> Size: 8B
array(1.2)
[999.     1.24   1.38   1.49   1.61   1.72   1.82   1.94   2.03   2.14
   2.24   2.36   2.46   2.57   2.68   2.8    2.92 999.  ]
[999.     1.24   1.38   1.49   1.61   1.72   1.82   1.94   2.03   2.14
   2.24   2.36   2.46   2.57   2.68   2.8    2.92 999.  ]
FINISHED 1 scenario, multiple indicators
0.19792938232421875


/home/byers/venvs/seturime/lib/python3.10/site-packages/pyam/core.py:2692: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  index = pd.unique(index)
/home/byers/venvs/seturime/lib/python3.10/site-packages/pyam/core.py:2692: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  index = pd.unique(index)
/home/byers/rime/rime/rime_functions.py:320: UserWarning: rename 'gwl' to 'year' does not create an index anymore. Try using swap_dims instead or use set_index after rename to create an indexed coordinate.
  updated_data = updated_data.rename({"gwl":"year"})


### Data is now transformed to 5 year interval starting 2015

In [18]:
map_out_MI

<xarray.Dataset> Size: 75MB
Dimensions:     (lat: 360, lon: 720, year: 18)
Coordinates:
  * lon         (lon) float32 3kB -179.8 -179.2 -178.8 ... 178.8 179.2 179.8
  * lat         (lat) float32 1kB 89.75 89.25 88.75 ... -88.75 -89.25 -89.75
  * year        (year) int64 144B 2015 2020 2025 2030 ... 2085 2090 2095 2100
Data variables:
    dri_qtot    (lat, lon, year) float64 37MB dask.array<chunksize=(360, 720, 1), meta=np.ndarray>
    iavar_qtot  (lat, lon, year) float64 37MB dask.array<chunksize=(360, 720, 1), meta=np.ndarray>
Attributes:
    model:     GCAM 5.3
    scenario:  NGFS2_Current Policies

## Plot dashboard (takes some minutes)

In [45]:
filename = "test_map_notebook.html"
plot_maps_dashboard(
    map_out_MI,
    filename=filename,
    year=2063,
    cmap="magma_r",
    shared_axes=True,
    clim=None,
)
#os.startfile(filename) # Windows
import webbrowser
webbrowser.open(f'file://{os.path.realpath(filename)}')

False

## Process corresponding table data, aggregated by country

### Import the climate impacts database files
Here is an example for sdii - standard precipitation index, and multiple variables, e.g.  
Exposure of land area  
Exposure of population  
Hazard value  
...  


In [46]:
filesall = glob.glob(fname_input_climate)
files = filesall  # [:6]
f = files[-0]
# load input climate impacts data file
ds = xr.open_mfdataset(f)
ds = ds.sel(year=years)
ds

IndexError: list index out of range

In [ ]:
varis = list(ds.data_vars.keys())[:lvaris]
dsi = ds[varis]
print(f"# of variables = {len(varis)}")

### Select scenarios data
Load an IAMC scenarios dataset (done previously at the beginning).  
Decide whether using global mean temperature or CO2 mode.  
Assign SSPs if missing and fix duplicate temperatures. 

In [ ]:
mode = "GMT"
if mode == "GMT":
    dfp = df_scens_in.filter(variable=temp_variable)
elif mode == "bypass":
    dfp = prepare_cumulative(df_scens_in, years=years, use_dask=True)
    ts = dfp.timeseries().apply(co2togmt_simple)
    ts = pyam.IamDataFrame(ts)
    ts.rename(
        {
            "variable": {ts.variable[0]: "RIME|GSAT_tcre"},
            "unit": {ts.unit[0]: "°C"},
        },
        inplace=True,
    )
    # Export data to check error and relationships
    # ts.append(dfp).to_csv('c://users//byers//downloads//tcre_gmt_output.csv')
    dfp = ts
    dfp.meta = df_scens_in.meta.copy()
dfp = dfp.filter(year=years)

In [ ]:
# For testing on C1 scenarios
# few_scenarios = True
# very_few_scenarios = False

# if few_scenarios:
#     dfp = dfp.filter(Category=["C1*"])
#     if very_few_scenarios:
#         dfp = dfp.filter(model="REMIND 2.1*", scenario="*")

dfp = ssps_imp

In [ ]:
# pre-prepare the dataset into correct format
#  Assign SSP to meta and select SSP2 in case SSP not present in name
dfp = ssp_helper(dfp, ssp_meta_col="Ssp_family", default_ssp="SSP2", keep_meta=False)

dft = dfp.timeseries()
dft = dft.join(dfp.meta["Ssp_family"]).reset_index()
# dft = dft.apply(fix_duplicate_temps, years=years, axis=1)

### START PROCESSING 

In [ ]:
start = time.time()
year_res = 10
parallel = True
if parallel:
    """
    For parallel processing, convert dft as a wide IAMC pd.Dataframe
    into a dask.DataFrame.
    """
    ddf = dd.from_pandas(dft, npartitions=1000)

    # dfx = dft.iloc[0].squeeze()  # FOR DEBUIGGING THE FUNCTION
    outd = ddf.apply(
        table_impacts_gmt,
        dsi=dsi,
        ssp_meta_col="Ssp_family",
        axis=1,
        meta=("result", None),
    )

    with ProgressBar():
        # try:
        df_new = outd.compute(num_workers=num_workers)
else:
    df_new = dft.apply(table_impacts_gmt, dsi=dsi, axis=1)

expandeddGMT = pd.concat([df_new[x] for x in df_new.index])
print(f" Done:  {time.time()-start}")

filename = f"RIME_output_{region}_{year_res}yr.csv"

# expandedd.to_csv(filename, encoding="utf-8", index=False)
print(f" Saved: {region} yrs={year_res}\n  {time.time()-start}")
print(f"{len(dsi.data_vars)} variables, {len(dfp.meta)} scenarios")

To see the Dask dasboard, open http://localhost:8787/status in your browser

In [ ]:
expandeddGMT = pyam.IamDataFrame(expandeddGMT)

In [ ]:
expandeddGMT.variable

In [ ]:
# model = "IMAG*"
scenario = [
    "CO_Bridge",
    "EN_INDCi2030_3000f",
    "EN_NPi2020_400f_lowBECCS",
    # 'NGFS2_Current Policies',
    "SSP1-26",
    "SSP2-45",
    # 'SSP3-Baseline',
    # 'SSP5-Baseline',
    "SusDev_SDP-PkBudg1000",
]
variable = "RIME|sdii|Hazard|Risk score|Population weighted"
# variable = 'RIME|wsi|Exposure|Population|%'
variable = "RIME|cdd|Exposure|Population|%"
expandeddGMT.filter(variable=variable, scenario=scenario, region="GBR").plot()

In [ ]:
expandeddGMT.filter(variable=variable, scenario=scenario, region="PAK").plot()

In [ ]:
expandeddGMT.variable

### Make SSP & IMP plots for presentation

In [ ]:
imps.scenario

In [ ]:
color_map = {
    "SSP1-26": "AR6-SSP1-2.6",
    "SSP2-45": "AR6-SSP2-4.5",
    "SSP3-Baseline": "AR6-SSP3-7.0",
    "SSP5-Baseline": "AR6-SSP5-8.5",
    "NGFS2_Current Policies": "AR6-IP-CurPol",
    # 'EN_NPi2020_900f': 'AR6-IMP-Neg2.0',
    "EN_INDCi2030_3000f": "AR6-IP-ModAct",
    # 'SSP2_openres_lc_50': 'AR6-IMP-Ren2.0',
    # AR6-IMP-GS',
    "CO_Bridge": "AR6-IMP-GS",
    "EN_NPi2020_400f_lowBECCS": "AR6-IMP-Neg",
    "SusDev_SDP-PkBudg1000": "AR6-IMP-SP",
}

pyam.run_control().update({"color": {"scenario": color_map}})

ssps_imp.filter(
    variable="AR6 climate diagnostics|Surface Temperature (GSAT)|MAGICCv7.5.3|50.0th Percentile",
    scenario="SSP*",
).plot(legend=False, fill_between=True, color="scenario", title="")

In [ ]:
ssps_imp.filter(
    variable="AR6 climate diagnostics|Surface Temperature (GSAT)|MAGICCv7.5.3|50.0th Percentile"
).plot(legend=False, fill_between=True, color="scenario", title="")